<div style="background-image: linear-gradient(to right, #4b4cff , #00d4ff); text-align: center; padding: 50px;">
  <h1 style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 48px; color: white; text-shadow: 2px 2px #4b4cff;">
    Projet - Deep learning
  </h1>
</div>

In [8]:
import os
from PIL import Image

def count_images_with_resolution(dataset_path, min_width=32, min_height=32):
    count = 0
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                try:
                    with Image.open(os.path.join(root, file)) as img:
                        width, height = img.size
                        if width >= min_width and height >= min_height:
                            count += 1
                except Exception as e:
                    print(f"Erreur avec le fichier {file}: {e}")
    return count

# Chemin du dossier 'test'
test_path = 'SF-MASK'

image_count = count_images_with_resolution(test_path)
print(f"Nombre d'images dans 'test' avec une résolution de 32x32 ou plus : {image_count}")

Nombre d'images dans 'test' avec une résolution de 32x32 ou plus : 32390


In [9]:
import os

def count_total_images(dataset_path):
    count = 0
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                count += 1
    return count

test_path = 'SF-MASK'

total_images = count_total_images(test_path)
print(f"Nombre total d'images dans 'SF-MASK' : {total_images}")

Nombre total d'images dans 'SF-MASK' : 45826


In [21]:
import os
from PIL import Image
import plotly.graph_objects as go
from collections import Counter

def get_category_for_resolution(width, height, categories):
    max_dim = max(width, height)
    for category in categories:
        cat_width, cat_height = map(int, category.split('x'))
        if max_dim <= cat_width:
            return category
    return categories[-1]  # If the resolution exceeds the largest category, assign it to the last category

def count_images_by_resolution(dataset_path, categories):
    resolution_counts = Counter()
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        width, height = img.size
                        category = get_category_for_resolution(width, height, categories)
                        resolution_counts[category] += 1
                except Exception as e:
                    print(f"Error with the file {file_path}: {e}")
    return resolution_counts

dataset_path = 'SF-MASK'

categories = ["16x16", "24x24", "32x32", "40x40", "48x48", "56x56", "64x64"]
resolutions = count_images_by_resolution(dataset_path, categories)
counts = [resolutions[category] for category in categories]

# Create the Plotly chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=categories,
    y=counts,
    marker_color='lightcoral',  # Changed bar color to light coral
    text=counts,
    textposition='outside',
    name='Number of Images'
))

fig.update_layout(
    title="Number of Images by Resolution Category in SF-MASK",
    xaxis_title="Resolution Categories",
    yaxis_title="Number of Images",
    template='plotly_white',
    yaxis=dict(showgrid=True),
)

# Display the chart
fig.show()

In [13]:
import os
import plotly.graph_objects as go

def count_images_in_nested_folders(dataset_path):
    folder_counts = {}
    for root, dirs, _ in os.walk(dataset_path):
        for dir_name in dirs:
            folder_path = os.path.join(root, dir_name)
            count = sum(
                1 for file in os.listdir(folder_path)
                if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))
            )
            if count > 0:  # Include only if the folder contains images
                relative_path = os.path.relpath(folder_path, dataset_path)
                folder_counts[relative_path] = count
    return folder_counts

# Path to the SF-MASK folder
dataset_path = 'SF-MASK'

# Count images in nested folders
image_counts = count_images_in_nested_folders(dataset_path)

# Separate data for `train` and `test`
categories = list(image_counts.keys())
counts = list(image_counts.values())
colors = ['blue' if 'train' in category else 'green' for category in categories]

# Create a Plotly bar chart
fig = go.Figure()

for category, count, color in zip(categories, counts, colors):
    fig.add_trace(go.Bar(
        x=[category],
        y=[count],
        marker_color=color,
        text=[count],  # Add the count on top of the bar
        textposition="outside",
        textfont=dict(size=14, color="black")  # Style the text
    ))

fig.update_layout(
    title="Number of Images per Subfolder in SF-MASK (Train vs Test)",
    xaxis_title="Subfolders",
    yaxis_title="Number of Images",
    template="plotly_white",
    xaxis=dict(tickangle=-45),
    showlegend=False  # Remove legend since colors are self-explanatory
)

fig.show()

In [17]:
import os
import plotly.graph_objects as go

def count_images_in_nested_folders(dataset_path):
    folder_counts = {}
    for root, dirs, _ in os.walk(dataset_path):
        for dir_name in dirs:
            folder_path = os.path.join(root, dir_name)
            count = sum(
                1 for file in os.listdir(folder_path)
                if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))
            )
            if count > 0:  # Include only if the folder contains images
                relative_path = os.path.relpath(folder_path, dataset_path)
                folder_counts[relative_path] = count
    return folder_counts

# Paths to the datasets
train_dataset_path = 'AFDB_face_dataset'  # Path for the first dataset
test_dataset_path = 'AFDB_masked_face_dataset'  # Path for the second dataset

# Count images in the datasets
train_image_counts = count_images_in_nested_folders(train_dataset_path)
test_image_counts = count_images_in_nested_folders(test_dataset_path)

# Combine the data for both datasets
# We ensure each category (subfolder) is included in both datasets' counts
all_categories = set(train_image_counts.keys()).union(test_image_counts.keys())

# Separate data for plotting
categories = list(all_categories)
train_counts = [train_image_counts.get(category, 0) for category in categories]
test_counts = [test_image_counts.get(category, 0) for category in categories]

# Create a Plotly bar chart
fig = go.Figure()

# Add bars for the training dataset
fig.add_trace(go.Bar(
    x=categories,
    y=train_counts,
    name="AFDB_face_dataset",
    marker_color='blue',
    text=train_counts,  # Add the count on top of the bar
    textposition="outside",
    textfont=dict(size=14, color="black")  # Style the text
))

# Add bars for the test dataset
fig.add_trace(go.Bar(
    x=categories,
    y=test_counts,
    name="AFDB_masked_face_dataset",
    marker_color='green',
    text=test_counts,  # Add the count on top of the bar
    textposition="outside",
    textfont=dict(size=14, color="black")  # Style the text
))

fig.update_layout(
    title="Number of Images per Subfolder in RMFRD Datasets (Train vs Test)",
    xaxis_title="Subfolders",
    yaxis_title="Number of Images",
    template="plotly_white",
    xaxis=dict(tickangle=-45),
    barmode='group',  # Group bars for train and test
    showlegend=True
)

fig.show()